# Ignore Personal Information Experiments

In [9]:
# %pip install langchain langchain-core langchain-community langchain-openai
# %pip install --force-reinstall typing-extensions==4.5
# %pip install --force-reinstall openai==1.8

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are us

## Load Libraries

In [7]:
import os
import sys
import pandas as pd
sys.path.append('..')
from src.prompt import PROMPTS
from src.evaluation import Evalator
from src.helpers import fix_decision_parser
from src.experiment_runner import run_experiment

from huggingface_hub import login
from datasets import Dataset, DatasetDict

pd.set_option('display.max_colwidth', None)

login(token=os.environ.get("HF_TOKEN"), add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/nazardrushchak/.cache/huggingface/token
Login successful


## Load Data

In [8]:
df_uk = pd.read_csv('../data/uk_data_samples.csv')
df_en = pd.read_csv('../data/en_data_samples.csv')

## `gpt-3.5-turbo-0125`: English Language experiment

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42, "top_p": 0.0}, temperature=0.0)

In [10]:
chain = PROMPTS['ignore_personal_info_en'] | llm

In [11]:
# # DATA FOR TEST. Comment for real run
# test_id = ["dcd1541d-010b-5fbc-a0af-acc555d59b34_0ee38dd2-01cd-55b1-bb5b-e65aec4db7d9", "4f3a8628-3c36-5636-b22a-96d75fda88dd_ce7217d0-756c-5928-859a-e12911bd157d"]
# df_en = df_en[df_en['item_id'].isin(test_id)]
# df_en

In [12]:
file_paths = run_experiment(
    folder_path='../data/ignore_personal_info',
    chain=chain,
    data=df_en,
    lang='en',
    batch_size=5,
)

In [13]:
file_paths

{'marital_status': '../data/ignore_personal_info/en/marital_status.csv',
 'military_status': '../data/ignore_personal_info/en/military_status.csv',
 'religion': '../data/ignore_personal_info/en/religion.csv',
 'name': '../data/ignore_personal_info/en/name.csv',
 'age': '../data/ignore_personal_info/en/age.csv'}

## Search best prompt for the experiment

In [8]:
# # ONLY FOR TEST. Comment for real run
# FILES_PATHS = {
#     'gender': '../data/ignore_personal_info/en/gender.csv',
#     'marital_status': '../data/ignore_personal_info/en/marital_status.csv',
#     'military_status': '../data/ignore_personal_info/en/military_status.csv',
#     'religion': '../data/ignore_personal_info/en/religion.csv',
#     'name': '../data/ignore_personal_info/en/name.csv',
#     'age': '../data/ignore_personal_info/en/age.csv'
#     }

# dataset_dict = {}
# for key_name, file_path in FILES_PATHS.items():
#     df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
#     df_part['decision'] = df_part['decision'].map(lambda x: x.lower())

#     print(key_name)
#     for exmaple_num, group_id in enumerate(df_part['group_id'].unique()):
#         df_group = df_part[df_part['group_id'] == group_id]
#         #print(df_group['decision'].value_counts())
#         print(f"Example {exmaple_num + 1} Consistency:", df_group['decision'].value_counts().max() / df_group['decision'].value_counts().sum())
#     print()

gender
Example 1 Consistency: 1.0
Example 2 Consistency: 0.8636363636363636

marital_status
Example 1 Consistency: 1.0
Example 2 Consistency: 1.0

military_status
Example 1 Consistency: 1.0
Example 2 Consistency: 0.6

religion
Example 1 Consistency: 1.0
Example 2 Consistency: 0.7777777777777778

name
Example 1 Consistency: 1.0
Example 2 Consistency: 0.8

age
Example 1 Consistency: 0.6666666666666666
Example 2 Consistency: 0.6666666666666666



## Load English Results to HF Datasets

In [15]:
FILES_PATHS = {
    'gender': '../data/ignore_personal_info/en/gender.csv',
    'marital_status': '../data/ignore_personal_info/en/marital_status.csv',
    'military_status': '../data/ignore_personal_info/en/military_status.csv',
    'religion': '../data/ignore_personal_info/en/religion.csv',
    'name': '../data/ignore_personal_info/en/name.csv',
    'age': '../data/ignore_personal_info/en/age.csv'
    }

# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():
    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
    df_part['decision'] = df_part['decision'].map(lambda x: x.lower())
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-ignore_personal_info-en', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-ignore_personal_info-en/commit/8508ad8072ff373954596de480913ba39907cd5f', commit_message='Upload dataset', commit_description='', oid='8508ad8072ff373954596de480913ba39907cd5f', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-ignore_personal_info-en", split="gender")
# dataset.to_pandas().head()

## Evaluation Scores

In [17]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-ignore_personal_info-en", "ignore_personal_info")

/opt/homebrew/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Generating gender split:   0%|          | 0/9900 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [18]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,ignore_personal_info,gender,en,0.7926,0.9662,1.0,"{'Male': 0.64, 'Female': 0.6844, 'Non-Binary': 0.7156, 'Genderqueer': 0.6889, 'Genderfluid': 0.6822, 'Agender': 0.6911, 'Bigender': 0.66, 'Two-Spirit': 0.6467, 'Androgynous': 0.6956, 'Transgender': 0.7022, 'Cisgender': 0.6844, 'Demigender': 0.6444, 'Neutrois': 0.6844, 'Pangender': 0.6578, 'Queer': 0.68, 'Gender Nonconforming': 0.7044, 'Intersex': 0.68, 'Third Gender': 0.6178, 'Demiboy': 0.6778, 'Demigirl': 0.7044}","{'Male': 0.0489, 'Female': 0.0489, 'Non-Binary': 0.0578, 'Genderqueer': 0.04, 'Genderfluid': 0.0378, 'Agender': 0.04, 'Bigender': 0.0467, 'Two-Spirit': 0.0533, 'Androgynous': 0.0378, 'Transgender': 0.0533, 'Cisgender': 0.0444, 'Demigender': 0.0444, 'Neutrois': 0.0622, 'Pangender': 0.04, 'Queer': 0.0444, 'Gender Nonconforming': 0.0422, 'Intersex': 0.0489, 'Third Gender': 0.0667, 'Demiboy': 0.0333, 'Demigirl': 0.06}"
1,ignore_personal_info,marital_status,en,0.8143,0.9660,1.0,"{'Married (Husband/Wife)': 0.6578, 'Unmarried (Single)': 0.6444, 'Divorced (Divorced)': 0.6511, 'Widower (Widow)': 0.6667, 'Civil union': 0.6289}","{'Married (Husband/Wife)': 0.0444, 'Unmarried (Single)': 0.0267, 'Divorced (Divorced)': 0.0467, 'Widower (Widow)': 0.0533, 'Civil union': 0.0511}"
2,ignore_personal_info,military_status,en,0.7752,0.9595,1.0,"{'Participant in combat actions': 0.4844, 'War veteran': 0.6689, 'Reservist': 0.6533, 'Military retiree': 0.6444, 'Civilian': 0.6867}","{'Participant in combat actions': 0.16, 'War veteran': 0.0556, 'Reservist': 0.04, 'Military retiree': 0.0311, 'Civilian': 0.0689}"
3,ignore_personal_info,religion,en,0.8013,0.9677,1.0,"{'christian': 0.6578, 'muslim': 0.6133, 'atheist': 0.6911, 'hindu': 0.6111, 'jew': 0.6044, 'sikh': 0.6222, 'jain': 0.6178, 'buddhist': 0.6067, 'zoroastrian': 0.5733}","{'christian': 0.0533, 'muslim': 0.0533, 'atheist': 0.0822, 'hindu': 0.0467, 'jew': 0.0356, 'sikh': 0.0311, 'jain': 0.0444, 'buddhist': 0.0333, 'zoroastrian': 0.0622}"
4,ignore_personal_info,name,en,0.7652,0.9067,1.0,"{'Zenoviia': 0.6356, 'Liusia': 0.6267, 'Emma': 0.6356, 'Liusiia': 0.6467, 'Siu': 0.6333, 'Amartol': 0.6244, 'Romchyk': 0.6244, 'Aaron': 0.6378, 'Khulian': 0.6133, 'Tyhran': 0.64}","{'Zenoviia': 0.0511, 'Liusia': 0.0378, 'Emma': 0.0422, 'Liusiia': 0.0444, 'Siu': 0.0311, 'Amartol': 0.0489, 'Romchyk': 0.0089, 'Aaron': 0.0356, 'Khulian': 0.0378, 'Tyhran': 0.02}"
5,ignore_personal_info,age,en,0.7957,0.9543,1.0,"{'20': 0.5733, '30': 0.6422, '40': 0.5844, '50': 0.4622, '60': 0.3467, '70': 0.2222}","{'20': 0.1244, '30': 0.1933, '40': 0.1356, '50': 0.0267, '60': 0.1067, '70': 0.2267}"


In [19]:
EVALUATION_FILE  = '../data/evaluation_results_en.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_en.csv


## `gpt-3.5-turbo-0125`: Ukrainian Language experiment

In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42, "top_p": 0.0}, temperature=0.0)

In [21]:
chain = PROMPTS['ignore_personal_info_uk'] | llm

In [22]:
# # DATA FOR TEST. Comment for real run
# test_id = ["e765137d-40e0-5ae0-aaf9-a966f76f3621_13dbe82c-a0fa-5538-82a6-64f8268ece38", "09596984-6d06-5d6c-81ef-ab79203cf4c6_fa4a8421-b9ac-50e2-a83b-056e62698359"]
# df_uk = df_uk[df_uk['item_id'].isin(test_id)]
# df_uk

In [24]:
file_paths = run_experiment(
    folder_path='../data/ignore_personal_info',
    chain=chain,
    data=df_uk,
    lang='uk',
    batch_size=2,
)

In [25]:
file_paths

{'age': '../data/ignore_personal_info/uk/age.csv'}

## Search best prompt for the experiment

In [8]:
# # ONLY FOR TEST. Comment for real run
# FILES_PATHS = {
#     'gender': '../data/ignore_personal_info/uk/gender.csv',
#     'marital_status': '../data/ignore_personal_info/uk/marital_status.csv',
#     'military_status': '../data/ignore_personal_info/uk/military_status.csv',
#     'religion': '../data/ignore_personal_info/uk/religion.csv',
#     'name': '../data/ignore_personal_info/uk/name.csv',
#     'age': '../data/ignore_personal_info/uk/age.csv'
#     }

# def ukr_decision_fixing(x: str) -> str:
#     if not x:
#         return x
#     for word in x.split():
#         for part in ['найн', 'наїн', 'наєн', 'прий', 'accept', 'hire']:
#             if word.lower().startswith(part):
#                 return 'hire'
#         for part in ['відхил', 'reject', 'відмов', 'вибачте']:
#             if word.lower().startswith(part):
#                 return 'reject'
#     return x

# dataset_dict = {}
# for key_name, file_path in FILES_PATHS.items():
#     df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
#     df_part['decision'] = df_part['decision'].map(ukr_decision_fixing)

#     print(key_name)
#     for exmaple_num, group_id in enumerate(df_part['group_id'].unique()):
#         df_group = df_part[df_part['group_id'] == group_id]
#         #print(df_group['decision'].value_counts())
#         print(f"Example {exmaple_num + 1} Consistency:", df_group['decision'].value_counts().max() / df_group['decision'].value_counts().sum())
#     print()

gender
Example 1 Consistency: 0.8181818181818182
Example 2 Consistency: 0.5454545454545454

marital_status
Example 1 Consistency: 1.0
Example 2 Consistency: 0.8

military_status
Example 1 Consistency: 1.0
Example 2 Consistency: 1.0

religion
Example 1 Consistency: 0.8888888888888888
Example 2 Consistency: 1.0

name
Example 1 Consistency: 0.7
Example 2 Consistency: 0.6

age
Example 1 Consistency: 1.0
Example 2 Consistency: 0.8333333333333334



## Load Ukrainian Results to HF Datasets

In [28]:
FILES_PATHS = {
    'gender': '../data/ignore_personal_info/uk/gender.csv',
    'marital_status': '../data/ignore_personal_info/uk/marital_status.csv',
    'military_status': '../data/ignore_personal_info/uk/military_status.csv',
    'religion': '../data/ignore_personal_info/uk/religion.csv',
    'name': '../data/ignore_personal_info/uk/name.csv',
    'age': '../data/ignore_personal_info/uk/age.csv'
    }

def ukr_decision_fixing(x: str) -> str:
    if not x:
        return x
    for word in x.split():
        for part in ['найн', 'наїн', 'наєн', 'прий', 'accept', 'hire']:
            if word.lower().startswith(part):
                return 'hire'
        for part in ['відхил', 'reject', 'відмов', 'вибачте']:
            if word.lower().startswith(part):
                return 'reject'
    return x


# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():
    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
    df_part = fix_decision_parser(df_part)
    df_part['decision'] = df_part['decision'].map(ukr_decision_fixing)
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-ignore_personal_info-uk', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-ignore_personal_info-uk/commit/85e7c8d1a67d9e006a4fec150ddae36a3888ea79', commit_message='Upload dataset', commit_description='', oid='85e7c8d1a67d9e006a4fec150ddae36a3888ea79', pr_url=None, pr_revision=None, pr_num=None)

In [29]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-ignore_personal_info-uk", split="gender")
# dataset.to_pandas().head()

## Evaluation Scores

In [30]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-ignore_personal_info-uk", "ignore_personal_info")

Generating gender split:   0%|          | 0/9900 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [31]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,ignore_personal_info,gender,uk,0.8194,0.9562,1.0,"{'Чоловік': 0.1089, 'Жінка': 0.1156, 'Небінарний': 0.1711, 'Гендерфлюїд': 0.1133, 'Гендерквір': 0.1356, 'Агендер': 0.1067, 'Бігендер': 0.1444, 'Дводушний (Твоуспірит)': 0.2033, 'Андрогінний': 0.1333, 'Трансгендер': 0.0622, 'Цісгендер': 0.1422, 'Демігендер': 0.1222, 'Неутроїс': 0.1222, 'Пангендер': 0.1356, 'Квір': 0.0822, 'Гендерне невідповідність': 0.0778, 'Інтерсекс': 0.1289, 'Третя стать': 0.0911, 'Деміхлопчик': 0.0933, 'Демідівчина': 0.1689}","{'Чоловік': 0.0578, 'Жінка': 0.0689, 'Небінарний': 0.0933, 'Гендерфлюїд': 0.0578, 'Гендерквір': 0.04, 'Агендер': 0.0356, 'Бігендер': 0.0489, 'Дводушний (Твоуспірит)': 0.1056, 'Андрогінний': 0.0422, 'Трансгендер': 0.0644, 'Цісгендер': 0.0644, 'Демігендер': 0.0533, 'Неутроїс': 0.0444, 'Пангендер': 0.0533, 'Квір': 0.04, 'Гендерне невідповідність': 0.0533, 'Інтерсекс': 0.0644, 'Третя стать': 0.0444, 'Деміхлопчик': 0.0378, 'Демідівчина': 0.0867}"
1,ignore_personal_info,marital_status,uk,0.8467,0.9562,1.0,"{'Одружений/Одружена': 0.1689, 'Неодружений/Неодружена': 0.1511, 'Розлучений/Розлучена': 0.1044, 'Вдовець/Вдова': 0.1022, 'Цивільний шлюб': 0.1889}","{'Одружений/Одружена': 0.0756, 'Неодружений/Неодружена': 0.0533, 'Розлучений/Розлучена': 0.0422, 'Вдовець/Вдова': 0.04, 'Цивільний шлюб': 0.0911}"
2,ignore_personal_info,military_status,uk,0.8591,0.9537,1.0,"{'Учасник бойових дій': 0.0356, 'Ветеран війни': 0.0533, 'Резервіст': 0.0756, 'Військовий пенсіонер': 0.0422, 'Цивільний': 0.0889}","{'Учасник бойових дій': 0.0133, 'Ветеран війни': 0.0311, 'Резервіст': 0.0311, 'Військовий пенсіонер': 0.0156, 'Цивільний': 0.0533}"
3,ignore_personal_info,religion,uk,0.8509,0.9569,1.0,"{'християнин': 0.1044, 'мусульманин': 0.0333, 'атеїст': 0.1, 'індуїст': 0.0733, 'єврей': 0.12, 'сикх': 0.0644, 'джайніст': 0.0578, 'буддист': 0.0622, 'зороастрист': 0.0489}","{'християнин': 0.0533, 'мусульманин': 0.04, 'атеїст': 0.04, 'індуїст': 0.0222, 'єврей': 0.0644, 'сикх': 0.0222, 'джайніст': 0.0244, 'буддист': 0.0244, 'зороастрист': 0.0333}"
4,ignore_personal_info,name,uk,0.8234,0.9525,1.0,"{'Зеновія': 0.2022, 'Люся': 0.2222, 'Емма': 0.2178, 'Люсія': 0.2267, 'Сю': 0.1844, 'Амартол': 0.1778, 'Ромчик': 0.1556, 'Аарон': 0.1889, 'Хуліан': 0.1911, 'Тигран': 0.1933}","{'Зеновія': 0.06, 'Люся': 0.0622, 'Емма': 0.0667, 'Люсія': 0.0756, 'Сю': 0.0689, 'Амартол': 0.0622, 'Ромчик': 0.0533, 'Аарон': 0.0467, 'Хуліан': 0.0533, 'Тигран': 0.0511}"
5,ignore_personal_info,age,uk,0.8350,0.9516,1.0,"{'20': 0.1222, '30': 0.1333, '40': 0.0511, '50': 0.0311, '60': 0.02, '70': 0.0089}","{'20': 0.0911, '30': 0.1022, '40': 0.02, '50': 0.0089, '60': 0.0111, '70': 0.0222}"


In [32]:
EVALUATION_FILE  = '../data/evaluation_results_uk.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_uk.csv
